# Loading data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sadmansakib7/ecg-arrhythmia-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ecg-arrhythmia-classification-dataset


In [2]:
import pandas as pd
import numpy as np


import torch


from sklearn.preprocessing import StandardScaler

# from datasets import DatasetData Preprocessing

In [3]:
data = pd.read_csv("/kaggle/input/ecg-arrhythmia-classification-dataset/INCART 2-lead Arrhythmia Database.csv")
data


,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,I01,N,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,I01,N,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,I01,N,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,I01,VEB,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,I01,N,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175724,I75,VEB,205,254,0.167375,0.998450,-1.884460,-2.278752,-1.884460,3,...,-0.063313,31,8,44,5,-0.063313,0.026827,0.679694,1.798788,1.291884
175725,I75,N,254,231,0.067664,0.986674,-0.321804,-0.321804,-0.743924,2,...,-0.038867,5,3,24,16,-0.038867,-0.370870,-0.697097,-0.948059,-1.078708
175726,I75,N,231,229,-0.129754,0.746366,-0.303551,-0.303551,-0.875649,3,...,-0.113876,4,4,24,16,-0.113876,-0.113876,-0.452218,-0.738737,-0.917931
175727,I75,N,229,232,-0.133953,0.747789,-0.626746,-0.626746,-0.871080,2,...,0.087784,18,8,42,16,0.087784,0.107684,0.308452,0.572327,-0.093598


In [4]:
print(data.shape)

(175729, 34)


In [5]:
data.groupby("record")["type"].count()

record
I01    2755
I02    2672
I03    2450
I04    2422
I05    1774
       ... 
I71    1669
I72    2268
I73    1990
I74    2403
I75    2101
Name: type, Length: 75, dtype: int64

In [6]:
data["type"].value_counts()

type
N       153546
VEB      20000
SVEB      1958
F          219
Q            6
Name: count, dtype: int64

In [7]:
# replacing VEB, SVEB, F, Q with general class

def normal_or_not(ex):
    if ex["type"] == "N":
        ex["type"] = 0

    else:
        ex['type'] = 1
    return ex

data = data.apply(normal_or_not, axis=1)

data = data.drop("record", axis=1)
data

,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,0_pq_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,0,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,2,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,0,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,5,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,0,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,25,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,1,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,3,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,0,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,34,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175724,1,205,254,0.167375,0.998450,-1.884460,-2.278752,-1.884460,3,18,...,-0.063313,31,8,44,5,-0.063313,0.026827,0.679694,1.798788,1.291884
175725,0,254,231,0.067664,0.986674,-0.321804,-0.321804,-0.743924,2,10,...,-0.038867,5,3,24,16,-0.038867,-0.370870,-0.697097,-0.948059,-1.078708
175726,0,231,229,-0.129754,0.746366,-0.303551,-0.303551,-0.875649,3,8,...,-0.113876,4,4,24,16,-0.113876,-0.113876,-0.452218,-0.738737,-0.917931
175727,0,229,232,-0.133953,0.747789,-0.626746,-0.626746,-0.871080,2,10,...,0.087784,18,8,42,16,0.087784,0.107684,0.308452,0.572327,-0.093598


In [8]:
typ = data["type"].to_numpy()
typ

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
sscaler = StandardScaler()
features = sscaler.fit_transform(data.drop("type", axis=1))

features

array([[-0.55393466, -0.52165771,  0.15905283, ...,  0.98283272,
         0.98594293,  0.72866882],
       [-0.52152196, -0.50545587, -0.70814796, ...,  1.24883726,
         1.22063578,  0.96419123],
       [-0.50531561, -1.54237313,  0.36611556, ...,  0.97909108,
         1.19408189,  1.78940353],
       ...,
       [ 0.54809728,  0.51525955, -0.87844323, ..., -0.32696011,
        -0.71128087, -0.77387298],
       [ 0.51568458,  0.56386504, -0.90029476, ...,  1.53592747,
         1.93738333,  0.91233524],
       [ 0.56430363,  0.25603023, -1.02407779, ..., -0.35033574,
        -0.71443075, -0.75300624]])

In [14]:
len(features[0])

32

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class HeartAttackCNN(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super(HeartAttackCNN, self).__init__()

        self.conv1 = nn.Conv1d(1, 16, kernel_size=3)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3)

        # self.linear = nn.Linear(32, 2)

        def conv_pool_output(L_in, kernel_size=3, pool_size=2, layers=2):
            L = L_in
            for _ in range(layers):
                L = L - (kernel_size - 1)  # Conv1d
                L = L // pool_size        # MaxPool1d
            return L

        flattened_size = 32 * conv_pool_output(32)

        self.pool = nn.MaxPool1d(2)
        self.linear1 = nn.Linear(flattened_size, 32)
        self.linear2 = nn.Linear(32, 2)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)

        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)

        return x

model = HeartAttackCNN()

In [19]:
model

HeartAttackCNN(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=192, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [24]:
parsed_data = pd.DataFrame(features)
parsed_data["labels"] = typ
parsed_data

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,labels
0,-0.553935,-0.521658,0.159053,-0.315567,-0.279796,0.242635,0.600250,-0.246442,-0.850889,-0.442460,...,0.343452,-0.536593,-0.340254,-0.899366,0.679597,0.760343,0.982833,0.985943,0.728669,0
1,-0.521522,-0.505456,-0.708148,0.752848,-1.079002,0.635430,0.098786,-1.376412,-0.470678,-0.992473,...,0.444310,-0.275312,-0.070591,-0.692378,1.049952,1.129474,1.248837,1.220636,0.964191,0
2,-0.505316,-1.542373,0.366116,0.883585,-1.000499,0.720547,-1.481576,-1.093919,2.064066,-0.103990,...,2.259741,0.769812,1.614804,-0.899366,0.238135,0.602668,0.979091,1.194082,1.789404,0
3,-1.542522,0.547663,0.714668,0.217768,-1.104834,-0.151043,0.198253,-1.282248,-0.724152,-0.992473,...,1.049453,0.247250,0.401320,-1.313341,0.927430,1.067446,1.465054,1.845378,2.026271,1
4,0.548097,-0.521658,2.844158,0.768873,-1.084534,0.629432,0.017233,-1.376412,3.204700,0.234479,...,2.158884,-0.275312,1.008062,-0.899366,0.365061,0.839678,1.004589,1.051866,1.646338,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175724,0.126732,0.920305,0.667825,1.382711,-3.161992,-2.116194,-4.637313,-1.376412,1.176906,-0.103990,...,2.058026,0.116609,1.075478,-1.106353,0.538722,0.884233,2.445102,4.415129,3.746400,1
175725,0.920843,0.547663,0.148923,1.364223,-1.359425,0.331381,-1.544764,-1.470576,0.163008,-0.865547,...,-0.564263,-0.536593,-0.272839,1.170509,0.624941,-0.396290,-0.926671,-1.134161,-1.102750,0
175726,0.548097,0.515260,-0.878443,0.986948,-1.338370,0.354211,-1.901935,-1.376412,-0.090466,-0.950164,...,-0.665121,-0.405953,-0.272839,1.170509,0.360389,0.431191,-0.326960,-0.711281,-0.773873,0
175727,0.515685,0.563865,-0.900295,0.989182,-1.711185,-0.050013,-1.889544,-1.470576,0.163008,-0.865547,...,0.746881,0.116609,0.940646,1.170509,1.071631,1.144580,1.535927,1.937383,0.912335,0


In [25]:
train = parsed_data[:150000]
test = parsed_data[150000:]

In [28]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

/usr/local/lib/python3.12/dist-packages/datasets/table.py:719: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  return cls(pa.Table.from_pandas(*args, **kwargs))


In [31]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=32)

test_dataloader = DataLoader(test_dataset, batch_size=32)